In [2]:
# the library used
import numpy as np
import pandas as pd
import pysubgroup as ps
import re
from sklearn.feature_extraction import DictVectorizer
import time
import ast
import string 
import imblearn
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



## Helper Function

In [3]:
def tags_preprocess(tags):
    """
    input: tag string
    output: list of individual tags in the given tag string
    function: preprocess a single tag string 
    """
    #tags = tags.replace("'","")
    #tags = tags.replace(" ","")
    #tags = tags.replace("[","")
    #tags = tags.replace("]","")
    tags = str(tags)
    tags = tags.split(" ")
    tags = [x.lower() for x in tags]
    return tags

In [4]:
def ingredients_preprocess(df):
    """
    input: dataframe
    output: list of distinct ingredient list
    function: preprocess the ingredient columns and return a list of distinct ingredients
    """
    distinct_ingredients = []
    dataframe = df
    
    for i in range(len(dataframe)):
        ingredients = dataframe.iloc[i]['ingredient']
        
        r = re.compile('[A-Z]{1}[a-zA-Z]+')
        ingredients = str(ingredients)
        #ingredients = ''.join(i for i in ingredients if not i.isdigit())
        ingredients = ingredients.replace("'","")
        #ingredients = ingredients.replace(" ","")
        ingredients = ingredients.replace("[","")
        ingredients = ingredients.replace("]","")
        # remove text inside parentheses
        ingredients = re.sub(r'\([^())]*\)',"", ingredients)
        ingredients = ingredients.split(" ")
        ingredients = list(filter(r.match, ingredients))
        ingredients = [x.lower() for x in ingredients]
        distinct_ingredients += ingredients
        dataframe.at[i, 'ingredient'] = ingredients
        #dataframe.set_value(i, 'ingredient', ingredients)
        
    return [list(set(distinct_ingredients)),dataframe]

In [5]:
def get_recipe_countries(countries, data):
    """
    input: list of countries, dataframe
    output: selected dataframe whose recipes is from these countries
    function: select the rows in dataframe whose "tag" value contain one country tag
    """
    # add a new column class 
    drop_index = []
    for i in range(len(data)):
        tags = data.loc[i]["tags"]
        tags = tags_preprocess(tags)
        
        country_same =[l for l in countries if l in tags]
            
        if len(country_same) == 1:
            data.at[i, 'label'] = country_same[0]
        if len(country_same) == 0:
            drop_index.append(i)
        if len(country_same) > 1:
            drop_index.append(i)
            #data.at[i, 'label'] = 'overlap'
            
    # drop the columns which has no season tags
    data = data.drop(data.index[drop_index])
    return data

In [6]:
def convert_to_dict(arr):
    """
    Helper function to convect an array of ingredients to a dictionary
    """
    d={}
    for a in arr:
        d[a]=1
    return d

In [7]:
# extract comment user dataset from original dataset
def extract_com_user(data):
    """
    input: dataframe
    output: a new dataframe with all the comment user information
    function: spilt the dictionary of the column 'comment user' in original dataset
    """
    df_com = pd.DataFrame()
    for index, item in data['comment_user'].iteritems():
        if (item != '[]'):
            if (item != 'no comment'):
                array = ast.literal_eval(item)
                df_array = pd.DataFrame(array)
                df_array['recipe_id'] = index
                df_com = pd.concat([df_com,df_array])
    return df_com

In [8]:
def sub_cat_in_com(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    punct = set(string.punctuation) 
    list_sub_cat = []
    
    df_com2 = pd.DataFrame()
    for index, item in data['calorie_value'].iteritems(): 
        if (item != None):
            list_sub = list(item)
            list_sub = ''.join(x for x in list_sub if x not in punct)
            list_sub_cat.append(list_sub)
    df_sub_cat = pd.DataFrame(list_sub_cat)
    df_sub_cat['calorie_value'] = df_sub_cat
    
    df_sub_cat['recipe_id'] = data['calorie_value'].index     
    df_com2 = pd.concat([df_com2,df_sub_cat])
    return df_com2

In [9]:
def add_recipe_info(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    punct = set(string.punctuation) 
    # add recipe name in comment users data
    list_recipe_name = []
    df_recipe = pd.DataFrame()
    for index, item in data['recipe_name'].iteritems(): 
        if (item != None):
            list_name = list(item)
            list_name = ''.join(x for x in list_name if x not in punct)
            list_recipe_name.append(list_name)

    df_name = pd.DataFrame(list_recipe_name)
    df_name['recipe_id'] = data['recipe_name'].index 
    df_name = df_name.set_index(["recipe_id"])
    df_name
    df_recipe['recipe_name'] = df_name[0]

    # add recipe difficulty in comment users data
    list_recipe_diff = []
    df_recipe_diff = pd.DataFrame()
    for index, item in data['difficulty'].iteritems(): 
        if (item != None):
            list_diff = list(item)      
            list_diff = ''.join(x for x in list_diff if x not in punct)   
            list_recipe_diff.append(list_diff)

    df_diff = pd.DataFrame(list_recipe_diff)
    df_diff['recipe_id'] = data['difficulty'].index 
    df_diff = df_diff.set_index(["recipe_id"])
    df_recipe['difficulty'] = df_diff[0]

    # add recipe preparation_time in comment users data
    list_recipe_pre = []
    df_recipe_pre = pd.DataFrame()
    for index, item in data['preparation_time'].iteritems(): 
        if (item != None):
            list_pre = list(item)

            list_pre = ''.join(x for x in list_pre if x not in punct)

            list_recipe_pre.append(list_pre)

    df_pre = pd.DataFrame(list_recipe_pre)
    df_pre['recipe_id'] = data['preparation_time'].index 
    df_pre = df_pre.set_index(["recipe_id"])
    df_recipe['preparation_time'] = df_pre[0]

    # add recipe tags in comment users data
    list_recipe_tags = []
    df_recipe_tags = pd.DataFrame()
    for index, item in data['tags'].iteritems(): 
        if (item != None):
            list_tags = list(item)      
            list_tags = ''.join(x for x in list_tags if x not in punct)   
            list_recipe_tags.append(list_tags)


    df_tags = pd.DataFrame(list_recipe_tags)
    df_tags['recipe_id'] = data['tags'].index 
    df_tags = df_tags.set_index(["recipe_id"])
    df_recipe['tags'] = df_tags[0]

    # add recipe ingredient in comment users data
    list_recipe_ingredient = []
    df_recipe_ingredient = pd.DataFrame()
    for index, item in data['ingredient'].iteritems(): 
        if (item != None):
            list_ingredient = list(item) 
            list_ingredient = ''.join(x for x in list_ingredient if x not in punct)
            list_recipe_ingredient.append(list_ingredient)

    df_recipe_ingredient = pd.DataFrame(list_recipe_ingredient)
    df_recipe_ingredient['recipe_id'] = data['ingredient'].index 
    df_recipe_ingredient = df_recipe_ingredient.set_index(["recipe_id"])
    df_recipe['ingredient'] = df_recipe_ingredient[0]
    return df_recipe

In [10]:
def age_group(age):   
    """
    input: age value
    output: group description
    function: divide age value uinto 5 groups 
    """   
    bucket = None
    age = int(age)    
    if age <= 30:
        bucket = '<30 Jahre'     
    if age in range(30, 46):
        bucket = '30-45 Jahre'
    if age in range(45, 61):
        bucket = '45-60 Jahre'
    if age >= 61:
        bucket = '60+ Jahre'
    return bucket

In [11]:
def calorie_level(calorie):   
    """
    input: calorie value
    output: group description
    function: divide calorie value into 3 groups 
    """   
    bucket = None
    calorie = int(calorie)    
    if calorie <= 300:
        bucket = 'low_calorie'    
    if calorie in range(300, 500):
        bucket = 'medium_calorie'        
    if calorie >= 500:
        bucket = 'high_calorie'      
    return bucket

In [12]:
def remove_None(data, name):
    """
    Helper function to remove None value in one column
    """ 
    y = data[data[name] == 'None']
    index_n = y.index.tolist()
    data = data.drop(index = index_n)
    return data

In [13]:
def add_target(data, calorie_level):
    df_sub_group[country] = df_dum_car[calorie_level]
    return df_sub_group

In [14]:
def pre_time_group(pre_time):   
    pre_time = int(pre_time)    
    if pre_time <= 20:
        bucket = '<20 Min'    
    if pre_time in range(20, 31):
        bucket = '20-30 Min'                
    if pre_time >= 31:
        bucket = '30+ Min'
    return bucket

## Subgroup Discovery

In this section we will use subgroup discovery to explore the association rules between attributes

- why we choose subgroup discovery?

because we find out that subgroup discovery is quite powerful compared to other data mining techniques. As long as we set differnt target with different search space, we can use use subgroup discovery technique to dig almost all interesting pattern that we want to explore from the dataset. 


In [16]:
# read the data
# data = pd.read_csv("all_data.csv")
data = pd.read_csv("/Users/xujingjing/Desktop/2020 Sommer/praktikum/Data/all_data.csv")

FileNotFoundError: [Errno 2] File b'all_data.csv' does not exist: b'all_data.csv'

In [297]:
# extract the recipe id from recipe urls
list_cat_no = []
i = 0
for item in data['recipe_url']:
    list_cat_no.append(item.split('/')[4])
    
# add one column "recipe_id" into the dataset and set it as the index of dataset
data['recipe_id'] = list_cat_no
data = data.set_index(["recipe_id"])

## Association rules between comment user information and recipe attributes

- data imbalance: gender
- improve the quality of data (preprossing)
- sampling 
- improne quality of result: different search algorithmn and selectors choices
- if the quality is also very low, plot the t-SNE

- associations with "calorie" and comment user information

since the job has more than half non values, so here we only explore the the association between  marriage_status, gender, and age with recipe calorie

In [298]:
# extract the numerical value string in the column calorie 
pat = r"([-+]?\d*\.\d+|\d+)"
data["calorie_value"] = data["calorie"].str.extract(pat, flags=0, expand=True)

# drop all rows with nan value in both columns comment_user and calorie_value
# data_com = data.dropna(subset=["comment_user",'calorie_value'])
len(data_com)


19058

In [299]:
# expand the corresponding recipe data for each comment_user
#data_com = data_com[0:500]
# data_ingrent = ingredients_preprocess(data_com)
# data_ingrent
df_com_1 = extract_com_user(data_com)
df_com_2 = sub_cat_in_com(data_com)
df_com_3 = add_recipe_info(data_com)
df_com_new = df_com_1.merge(df_com_2, on='recipe_id', how='left')
df_com_new = df_com_new.merge(df_com_3, on='recipe_id', how='left')

df_com_new = df_com_new[['recipe_id','recipe_name','tags','difficulty','preparation_time','ingredient','name','rating','sex','age','marriage_status','comment_time','calorie_value']]
df_com_new = df_com_new.set_index(["recipe_id"])


KeyboardInterrupt: 

In [300]:
# df_com_new


,recipe_name,tags,difficulty,preparation_time,ingredient,name,rating,sex,age,marriage_status,comment_time,calorie_value,pretime_value,pre_time_group,calorie_level,age_value
recipe_id,,,,,,,,,,,,,,,,
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Haubndaucher,rating-5,männlich,46 Jahre,Verheiratet,10.08.2017 14:56,500,30,30-60 Min,high_calorie,46
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Lumakath,rating-4,weiblich,49 Jahre,Verheiratet,19.01.2013 13:19,500,30,30-60 Min,high_calorie,49
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,esther76,rating-4,weiblich,None,Verheiratet,21.10.2012 17:56,500,30,30-60 Min,high_calorie,NaN
2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,t1n4a,rating-1,weiblich,25 Jahre,None,13.01.2020 21:05,205,5,<30 Min,low_calorie,25
2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,patty89,rating-4,weiblich,None,Verheiratet,03.09.2019 13:18,205,5,<30 Min,low_calorie,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148171468749120,Übernacht Weizenbrötchen,Backen Vegetarisch Frühstück BrotoderBrötchen Gluten Lactose,normal,30 Min,500g WeizenmehlType550 300ml Wasser 1ELgehäuft Butterweiche 2TLgestr Salz 5g Hefefrisch Sesamode...,tig,rating-5,weiblich,None,None,22.12.2017 16:42,493,30,30-60 Min,medium_calorie,NaN
3148171468749120,Übernacht Weizenbrötchen,Backen Vegetarisch Frühstück BrotoderBrötchen Gluten Lactose,normal,30 Min,500g WeizenmehlType550 300ml Wasser 1ELgehäuft Butterweiche 2TLgestr Salz 5g Hefefrisch Sesamode...,Kuchlmasta,rating-5,männlich,65 Jahre,None,18.07.2016 18:47,493,30,30-60 Min,medium_calorie,65
3148171468749120,Übernacht Weizenbrötchen,Backen Vegetarisch Frühstück BrotoderBrötchen Gluten Lactose,normal,30 Min,500g WeizenmehlType550 300ml Wasser 1ELgehäuft Butterweiche 2TLgestr Salz 5g Hefefrisch Sesamode...,Fee56,rating-5,None,None,None,18.07.2016 11:09,493,30,30-60 Min,medium_calorie,NaN


In [1]:
# remove none value in the whole data set 
df_com_no_none = df_com_new.mask(df_com_new.astype(object).eq('None')).dropna()
# df_com_new = df_com_no_none
df_com_no_none = df_com_no_none.reset_index()
# df_com_no_none

NameError: name 'df_com_new' is not defined

In [399]:
# add calorie level , preparation time and age group columns in the comment user information
df_com_no_none["pretime_value"] = df_com_no_none["preparation_time"].str.extract(pat, flags=0, expand=True)
df_com_no_none['pre_time_group'] = df_com_no_none['pretime_value'].apply(pre_time_group)

df_com_no_none['calorie_level'] = df_com_no_none['calorie_value'].apply(calorie_level)
df_dum_calorie = pd.get_dummies(df_com_no_none['calorie_level'])

df_com_no_none["age_value"] = df_com_no_none["age"].str.extract(pat, flags=0, expand=True)
df_com_no_none['age_group'] = df_com_no_none['age_value'].apply(age_group)

df_com_dum_calorie = df_com_no_none.join(df_dum_calorie, how='left')
df_com_dum_calorie['age_group'].value_counts()



30-45 Jahre    43292
45-60 Jahre    26865
60+ Jahre       7498
<30 Jahre       7418
Name: age_group, dtype: int64

- the target is calorie and features are the sex,age,marriage status of comment users 

In [400]:
data = df_com_dum_calorie[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
# data = data.dropna(axis=0, how="any")
print(len(data))
target = ps.BinaryTarget('low_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

85073
    quality                                   description
0  0.002450    age_group=='60+ Jahre' AND sex=='weiblich'
1  0.002306  age_group=='45-60 Jahre' AND sex=='weiblich'
2  0.002248                        age_group=='60+ Jahre'
3  0.002194                      age_group=='45-60 Jahre'
4  0.002010                               sex=='weiblich'


In [305]:
df_com_dum_calorie['calorie_level'].value_counts()

high_calorie      46732
medium_calorie    19498
low_calorie       18843
Name: calorie_level, dtype: int64

#### Deal with imbalanced data

"high_calorie": "medium_calorie": "low_calorie" ratio is around 2.5:1:1, the data is quite imblanced with regard to the calorie level, we want to try to resample the dataset to help improve the data quality. We want to see if we can improve the quality of subgroup discovery

- upsampling

In [401]:
df_sampling = df_com_dum_calorie[['rating','sex','age_group','marriage_status','calorie_level','pre_time_group','difficulty','tags','ingredient']]
# df_sampling.head()

In [307]:
# oversampling
# Separate input features and target
y = df_sampling['calorie_level']
X = df_sampling.drop('calorie_level', axis=1)

# setting up testing and training sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# concatenate our training data back together
# X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
high_calorie = df_sampling[df_sampling['calorie_level'] == 'high_calorie']
low_calorie = df_sampling[df_sampling['calorie_level'] == 'low_calorie']
medium_calorie = df_sampling[df_sampling['calorie_level'] == 'medium_calorie']


# upsample minority
low_upsampled = resample(low_calorie,
                          replace=True, # sample with replacement
                          n_samples=len(high_calorie), # match number in majority class
                          random_state=27) # reproducible results

medium_upsampled = resample(medium_calorie,
                          replace=True,
                          n_samples=len(high_calorie), 
                          random_state=27) 


# combine majority and upsampled minority
upsampled = pd.concat([high_calorie, low_upsampled])
upsampled = pd.concat([upsampled, medium_upsampled])
upsampled.reset_index(inplace = True)
upsampled['calorie_level'].value_counts()
# upsampled.head()

high_calorie      46732
low_calorie       46732
medium_calorie    46732
Name: calorie_level, dtype: int64

In [402]:
# add the dummyset into the upsampled dataset
upsampled_dum = pd.get_dummies(upsampled['calorie_level'])
upsampled_dum.reset_index(inplace = True)
upsampled_dum = upsampled_dum.drop('index', axis=1)

# prepare for the binary target columns
# upsampled = upsampled.drop('index', axis=1)
upsampled_new = upsampled.join(upsampled_dum, how='left')
# upsampled_new = upsampled_new.drop('calorie_level',1)
# upsampled_new['sex'].value_counts()


In [403]:
data = upsampled_new[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
target = ps.BinaryTarget('high_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                                  description
0  0.006294                                     age_group=='30-45 Jahre'
1  0.004969                 age_group=='30-45 Jahre' AND sex=='weiblich'
2  0.003074  age_group=='30-45 Jahre' AND marriage_status=='Verheiratet'
3  0.002340              marriage_status=='Vergeben' AND sex=='weiblich'
4  0.002325                                  marriage_status=='Vergeben'


- undersampling

In [404]:
# downsample majority
high_downsampled = resample(high_calorie,
                                replace = False, # sample without replacement
                                n_samples = len(low_calorie), # match minority n
                                random_state = 27) # reproducible results

medium_downsampled = resample(medium_calorie,
                                replace = False,
                                n_samples = len(low_calorie),
                                random_state = 27)

# combine minority and downsampled majority
downsampled = pd.concat([high_downsampled, low_calorie])
downsampled = pd.concat([downsampled, medium_downsampled])

# add the dummyset into the downsampled dataset
downsampled_dum = pd.get_dummies(downsampled['calorie_level'])
# downsampled_dum.reset_index(inplace = True)
# downsampled.reset_index(inplace = True)
downsampled_new = downsampled.join(downsampled_dum, how='left')
downsampled_new['calorie_level'].value_counts()


high_calorie      18843
low_calorie       18843
medium_calorie    18843
Name: calorie_level, dtype: int64

In [412]:
data = downsampled_new[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
target = ps.BinaryTarget('low_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                       description
0  0.002830                                   sex=='weiblich'
1  0.002689        age_group=='60+ Jahre' AND sex=='weiblich'
2  0.002282      age_group=='45-60 Jahre' AND sex=='weiblich'
3  0.002129  marriage_status=='Verwitwet' AND sex=='weiblich'
4  0.002129                            age_group=='60+ Jahre'


- Explore the male and female comment users separately

In [408]:
# df_male = df_com_dum_calorie[df_com_dum_calorie['sex'] == 'männlich']
# data = df_male[['age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
# target = ps.BinaryTarget('low_calorie', True)
# searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
# task = ps.SubgroupDiscoveryTask (
#     data, 
#     target, 
#     searchspace, 
#     result_set_size=5, 
#     depth=2, 
#     qf=ps.WRAccQF())
# result = ps.BeamSearch().execute(task)
# pd.set_option('max_colwidth',100)
# print(result.to_dataframe())

In [409]:
# df_female = df_com_dum_calorie[df_com_dum_calorie['sex'] == 'weiblich']
# data = df_female[['age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
# target = ps.BinaryTarget('low_calorie', True)
# searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
# task = ps.SubgroupDiscoveryTask (
#     data, 
#     target, 
#     searchspace, 
#     result_set_size=5, 
#     depth=2, 
#     qf=ps.WRAccQF())
# result = ps.BeamSearch().execute(task)
# pd.set_option('max_colwidth',100)
# print(result.to_dataframe())

#### Conclusion 

- the subgroup young female seems to prefer the "high-calorie" recipes
- the subgroup married middle-age male seems to prefer the "medium-calorie" recipes
- the subgroup divorced middle-age and older female seems to prefer the "low-calorie" recipes

then our next step is to discover why differnt calorie-level is preferred by these particular groups

### Existing problems
- Here we remove all rows which have nan, does it make sense to use other fill nan methods? 
- atfer resampling the quality is still low, how can we then to improve the quality?

- the target is calorie level and features is the preparation time and difficulty of recipes.

In [318]:
df_com_dum_calorie['pre_time_group'].value_counts()

20-30 Min    45209
<20 Min      28338
30+ Min      11526
Name: pre_time_group, dtype: int64

In [413]:
data = downsampled_new[['high_calorie', 'low_calorie', 'medium_calorie','pre_time_group','difficulty']]
target = ps.BinaryTarget('medium_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                           description
0  0.014641                                  difficulty=='normal'
1  0.013344  difficulty=='normal' AND pre_time_group=='20-30 Min'
2  0.007701                           pre_time_group=='20-30 Min'
3  0.002253    difficulty=='normal' AND pre_time_group=='<20 Min'
4  0.001285                                 difficulty=='pfiffig'


- the target is the attributes of users and features is the preparation time and difficulty of recipes.

In [415]:
new_data = downsampled_new[['sex','age_group','marriage_status','pre_time_group','difficulty','calorie_level']]

#  set the target value by combining 'sex', 'age_group', 'marriage_status'
new_data['combine'] = new_data[['sex', 'age_group', 'marriage_status']].values.tolist()
new_data['combine'] = new_data['combine'].astype(str)

# one hot embedding of target value
combine_dum = pd.get_dummies(new_data['combine'])
new_data = new_data.join(combine_dum, how='left')
# combine_dum
# new_data.head()

/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [397]:
# discover the relationship bewteen all the comment user groups and the preparation time and difficulty of recipes

data = new_data[['pre_time_group','difficulty']]
i =0
for col in combine_dum:
    column_name = col
    column_name = str(column_name)
    data[column_name] = new_data[column_name]

    target = ps.BinaryTarget(column_name, True)
    searchspace = ps.create_selectors(data, ignore=[column_name])
    task = ps.SubgroupDiscoveryTask (
        data, 
        target, 
        searchspace, 
        result_set_size=5, 
        depth=2, 
        qf=ps.WRAccQF())
    result = ps.BeamSearch().execute(task)
    pd.set_option('max_colwidth',100)
    result = result.to_dataframe()
    result = result.rename(columns={'description':column_name})
    data = data.drop(column_name,1)
    i = i+1
    print('----------------------------------------------------------------------')
    print("number of target combination is:", i)
    print(result)

/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


----------------------------------------------------------------------
number of target combination is: 1
    quality             ['männlich', '30-45 Jahre', 'Geschieden']
0  0.000042  difficulty=='normal' AND pre_time_group=='20-30 Min'
1  0.000026                                  difficulty=='normal'
2  0.000023                           pre_time_group=='20-30 Min'
3  0.000000                                               Dataset
----------------------------------------------------------------------
number of target combination is: 2
    quality                 ['männlich', '30-45 Jahre', 'Single']
0  0.000351                             pre_time_group=='<20 Min'
1  0.000297    difficulty=='simpel' AND pre_time_group=='<20 Min'
2  0.000136                                  difficulty=='normal'
3  0.000115  difficulty=='normal' AND pre_time_group=='20-30 Min'
4  0.000054    difficulty=='normal' AND pre_time_group=='<20 Min'
--------------------------------------------------------------

### Discover why differnt calorie-level is preferred by these particular groups

- set target as colorie-level/ subgroup, still use subgroup discovery

In [266]:
def get_distinct_tag(df):
    distinct_tags = []
    for i in range(len(df)):
        distinct_tags += df.iloc[i]['tags']
    return list(set(distinct_tags))

- set target as subgroup, explore the tags of recipes, use multiclass-classifier

In [322]:
downsampled_new = downsampled_new.reset_index()
downsampled_new['tags'] = downsampled_new['tags'].astype(object)
for i in range(len(downsampled_new)):
    tags = downsampled_new.iloc[i]['tags']
    tags = tags_preprocess(tags)
    downsampled_new.at[i, 'tags'] = tags

In [323]:
# prepare the data for multiclass classifier
df = downsampled_new[['tags', 'sex', 'age_group', 'marriage_status', 'calorie_level']]
distinct_tags = get_distinct_tag(df)

# One hot encoding of the ingredients
df['tags'] = df.tags.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df = pd.DataFrame(data = vectorizer.fit_transform(df.tags.tolist()), columns = distinct_tags)
new_df[['sex', 'age_group', 'marriage_status', 'calorie_level']] = df[['sex', 'age_group', 'marriage_status', 'calorie_level']]

# set the target value by combining 'sex', 'age_group', 'marriage_status'
new_df['combine'] = new_df[['sex', 'age_group', 'marriage_status']].values.tolist()
new_df['combine'] = new_df['combine'].astype(str)
new_df

/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,aufstrich,rind,brotoderbrötchen,braten,suppe,informempfehlung,mexiko,reisodernudelsalat,amerika,dünsten,...,eis,osteuropa,tee,lammoderziege,grillen,sex,age_group,marriage_status,calorie_level,combine
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,<30 Jahre,Single,high_calorie,"['weiblich', '<30 Jahre', 'Single']"
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,60+ Jahre,Verheiratet,high_calorie,"['weiblich', '60+ Jahre', 'Verheiratet']"
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,45-60 Jahre,Verheiratet,high_calorie,"['weiblich', '45-60 Jahre', 'Verheiratet']"
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,weiblich,45-60 Jahre,Verheiratet,high_calorie,"['weiblich', '45-60 Jahre', 'Verheiratet']"
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,30-45 Jahre,Single,high_calorie,"['weiblich', '30-45 Jahre', 'Single']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56524,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,weiblich,45-60 Jahre,Vergeben,medium_calorie,"['weiblich', '45-60 Jahre', 'Vergeben']"
56525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,45-60 Jahre,Verheiratet,medium_calorie,"['weiblich', '45-60 Jahre', 'Verheiratet']"
56526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,45-60 Jahre,Verheiratet,medium_calorie,"['weiblich', '45-60 Jahre', 'Verheiratet']"
56527,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,weiblich,30-45 Jahre,Verheiratet,medium_calorie,"['weiblich', '30-45 Jahre', 'Verheiratet']"


In [271]:
# multiclass classifer
clf_X = new_df.iloc[:, :-5]
y = new_df.iloc[:, -1]

clf = LogisticRegression(max_iter=1000)
clf.fit(clf_X, y)
y_pred = clf.predict(clf_X)

# accuracy 
print("Accuracy:",clf.score(clf_X, y))

Accuracy: 0.18733747280157087


In [ ]:
def computeIDF(clf_X):
    num_document = len(clf_X)
    idf_row = clf_X.sum(axis = 0, skipna = True) 
    idf_row = len(clf_X)/idf_row
    return idf_row

def computeTF(clf_X):
    clf_X['sum'] = clf_X.sum(axis = 1, skipna = True)
    clf_X = clf_X.div(clf_X['sum'], axis=0)
    clf_X = clf_X.drop(['sum'],axis = 1)
    return clf_X

In [272]:
# use td-idf to preprocess the data
idf_row = computeIDF(clf_X)
clf_X = computeTF(clf_X)
clf_X = clf_X*idf_row
clf_X = clf_X.fillna(0)

NameError: name 'computeIDF' is not defined

In [273]:
# multiclass classifier after preprocessing with td-idf
clf = LogisticRegression(max_iter=1000)
clf.fit(clf_X, y)
y_pred = clf.predict(clf_X)

# accuracy 
print("Accuracy:",clf.score(clf_X, y))

Accuracy: 0.18733747280157087


In [293]:
df = pd.DataFrame()
for i in range(len(clf.classes_)):
    if clf.classes_[i] != 'overlap':
        df[clf.classes_[i]] = [vectorizer.feature_names_[ing_id] for ing_id in np.argsort(-clf.coef_[i])[:10]]
a = ['weiblich', '35-45 Jahre', 'Verheiratet']
# a = ['männlich', '55+ Jahre', 'Verheiratet']
a = str(a)
df[a]


KeyError: "['weiblich', '35-45 Jahre', 'Verheiratet']"

    quality                                         description
0  0.002353                                     sex=='männlich'
1  0.001238          age_group=='55+ Jahre' AND sex=='männlich'
2  0.001067     marriage_status=='Vergeben' AND sex=='männlich'
3  0.001056  marriage_status=='Verheiratet' AND sex=='männlich'
4  0.001020                            age_group=='45-55 Jahre'

- set target as subgroup, explore the ingredients of recipes, use multiclass-classifier

In [284]:
# prepare the data for multi-class classifier
distinct_ingredients, df_2 = ingredients_preprocess(downsampled_new)

# One hot encoding of the ingredients
df_2['ingredient'] = df_2.ingredient.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df_2 = pd.DataFrame(data = vectorizer.fit_transform(df_2.ingredient.tolist()), columns = distinct_ingredients)
new_df_2[['sex', 'age_group', 'marriage_status', 'calorie_level']] = df_2[['sex', 'age_group', 'marriage_status', 'calorie_level']]

# set the target value by combining 'sex', 'age_group', 'marriage_status'
new_df_2['combine'] = new_df_2[['sex', 'age_group', 'marriage_status']].values.tolist()
new_df_2['combine'] = new_df_2['combine'].astype(str)

In [285]:
# multiclass classifer
clf_X = new_df_2.iloc[:, :-5]
y = new_df_2.iloc[:, -1]

clf = LogisticRegression(max_iter=1000)
clf.fit(clf_X, y)
y_pred = clf.predict(clf_X)

# accuracy 
print("Accuracy:",clf.score(clf_X, y))

Accuracy: 0.31323745334253217


In [288]:
df = pd.DataFrame()
for i in range(len(clf.classes_)):
    if clf.classes_[i] != 'overlap':
        df[clf.classes_[i]] = [vectorizer.feature_names_[ing_id] for ing_id in np.argsort(-clf.coef_[i])[:20]]
df


,"['männlich', '35-45 Jahre', 'Geschieden']","['männlich', '35-45 Jahre', 'Single']","['männlich', '35-45 Jahre', 'Vergeben']","['männlich', '35-45 Jahre', 'Verheiratet']","['männlich', '45-55 Jahre', 'Geschieden']","['männlich', '45-55 Jahre', 'Single']","['männlich', '45-55 Jahre', 'Vergeben']","['männlich', '45-55 Jahre', 'Verheiratet']","['männlich', '45-55 Jahre', 'Verwitwet']","['männlich', '55+ Jahre', 'Geschieden']",...,"['weiblich', '55+ Jahre', 'Geschieden']","['weiblich', '55+ Jahre', 'Single']","['weiblich', '55+ Jahre', 'Vergeben']","['weiblich', '55+ Jahre', 'Verheiratet']","['weiblich', '55+ Jahre', 'Verwitwet']","['weiblich', '<35 Jahre', 'Geschieden']","['weiblich', '<35 Jahre', 'Single']","['weiblich', '<35 Jahre', 'Vergeben']","['weiblich', '<35 Jahre', 'Verheiratet']","['weiblich', '<35 Jahre', 'Verwitwet']"
0,kartoffeln,sahnegeschlagen,kartoffelnmöglichstgleichgroßeundmehligkochend,schweinefleischlachsbraten,sahnegeschlagen,toastbrotweißbrot,brötchen,mehlzumbestäubenderarbeitsfläche,butterjenachgeschmack,fischfilets,...,wasserlauwarmes,buttergeschmolzene,topinambur,zwetschgenoderpflaumen,joghurtgriechischer,lachstk,schokoladenachwahl,kräuteritalienischetk,spekulatius,margarinevegane
1,karotten,chilischotenrote,kräuterlingezumstreuen,chiliflockenodercayennepfeffer,chilischotenrot,goudamittelalter,backpulverodertrockenhefe,fischfiletsmatjes,wein,gewürzgurken,...,pflanzenöl,getränkepulverovomaltine,hirse,speckwürfel,thunfischineigenemsaft,weißkohl,baconoderschinkenspeck,kondensmilch10,saftkarotten,vollkornweizenmehl
2,bacongewürfelt,sauerteig,limetten,kakao,puderzucker,majorangerebbelt,honigmelonen,bandnudelnanderegehennatürlichauch,rapsöl,muskat,...,puderzucker50gbuntezuckerstreusel,kuvertüreweiße,fleischbrüheklare,kondensmilchmilchmädchengezuckerte,kartoffelnkleinedrillinge,chinakohl,brokkolifrischer,buttervanillearomaoptional,salatgurkenodereinlegegurkenoderzucchini,wasserfürdielauge
3,sesam,sekt,speckgeräuchert,rum54,naturjoghurt,ingwer,sellerie,handkäse,lauch,olivenschwarze,...,kokosblütenzucker,ingwerfrisch,gewürzmischungchinagewürz,wasserkaltesoder1ei,petersilieoderschnittlauch,puddingpulvervanillegeschmack,majorangerebelt,eisvanilleeisevtl,speckdurchwachsenundgeräuchert,salzgroboderfeinzumbestreuen
4,tomatenmark,rinderhackfleisch,kartoffelnfestkochendgernegrößere,bourbonvanillezucker,margarinereformhaus,tomatenfeste,eierleichtverquirlt,frühstücksspeckhauchdünngeschnitten,champignons,apfelinkleinenstücken,...,erdbeerenoderalternativhimbeeren,zwiebelnkleingewürfelt,salatrömersalat,butteroderöl,sardellenfilets,reislangkornreis,kräuterfrischenachbeliebenschnittlauchpetersiliekressedillliebstöckeletc,paprikarotundgelb,zuckerodersüßstoffnachwahl,weizenmehlvollkorn
5,kasseler,olivenschwarzohnestein,rosinenodersultaninen,honigflüssigen,senfsüßer,bier,butterinflöckchen,weizenmehl550,thymian,vanilleschotendasmark,...,anstellgut,kirschsaft,bohnenweiße,haferflockenkernig,heidelbeerenblaubeeren,miracelwhip,naturjoghurtfettarm150g,gewürzgurkengrglas,beerentkungezuckertaufgetaut,natron
6,gemüsefond,maismaiskörneràca280geinwaage,fettzumfrittieren,lachsgeräucherter,kartoffelnmehligegeschälte,saucenbinderbrauner,tomaten400g,bierje05lzbveltins,algengrüneausderfischtheke,broterindetrockenevondunklembrotzerbröckelt,...,joghurttürkischensüzme,estragongetrocknet,semmelnaltbackene,stachelbeeren,pellkartoffelnvomvortag,zwiebelninringegeschnitten,melonengaliamelonedasfruchtfleischabwiegen,zimtganz,saucetexicanasalsasauce,meersalz
7,hefe,blattspinat,kuvertüre,zwiebelnlila,mett,lebernvomschweine,eiervomhuhnweißoderbraun,himbeerentkca300350g,feigensenf,kondensmilchdosenmilchkaffeesahne,...,knoblauchzehengeschält,pistaziengehackte,stärkemehl,kartoffelnsalatkartoffeln,olivenschwarzeentsteint,früchtefrischodertkerdbeerenhimbeerenoderbrombeeren,gnocchiausdemkühlregal,zitronenunbehandeltedensaftdavonundvoneinerdieabgeriebeneschale,cranberriesgetrocknet,hefefrisch
8,kartoffelngeviertelt,rinderbraten,schweineschnitzeláca150g,tortellinif

- set target as calorie, explore the tags of recipes, use multiclass-classifier

In [289]:
# multiclass classifer
clf_X = new_df.iloc[:, :-5]
y = new_df.iloc[:, -2]

clf = LogisticRegression(max_iter=1000)
clf.fit(clf_X, y)
y_pred = clf.predict(clf_X)

# accuracy 
print("Accuracy:",clf.score(clf_X, y))

Accuracy: 0.6493481222027632


In [290]:
df = pd.DataFrame()
for i in range(len(clf.classes_)):
    if clf.classes_[i] != 'overlap':
        df[clf.classes_[i]] = [vectorizer.feature_names_[ing_id] for ing_id in np.argsort(-clf.coef_[i])[:10]]
df

,high_calorie,low_calorie,medium_calorie
0,ananasringe,apfelmusgehtauchohne,amarantpops
1,apfelmusevtlselbstgemacht,apfelmusausdemglas,apfelessigoderweinessig
2,ananasoderfrische,acetobalsamicohell,agavendicksaftoderhonig
3,ananasfrisch,apfelinstücken,apfelgehackt
4,aalegeräuchert,amarettoca,ananassaftdose
5,apfelca80g,acetobalsamicoacetodimodena,apfelentkerntundkleingewürfelt
6,apfelmus,anisgemahlen,ananasinstückenausderdose
7,anstellgut,ajvarscharfodermild,anstellgut
8,anissternanis,ananasinstücken,acetobalsamicoacetodimodena
9,ajvarpaprikapasteausdemglas,ajvaroderpikantetomatensauce,ananasinstücken


### Association rules among the recipe attributes

- the association with "ingredient" and "regions"

In [ ]:
countries = ['china', 'indien', 'deutschland','frankreich','grossbritannien','österreich','usaoderkanada','italien','spanien',
             'portugal', 'japen','schweiz','türkei', 'thailand', 'russland', 'großbritannien & irland', 'vietnam', 'korea',
            'australien', 'ägypten', 'marokko', 'niederlande']

In [ ]:
df_certain_countries = get_recipe_countries(countries, data)
df_certain_countries.reset_index(inplace = True)
df_certain_countries['label'].value_counts()

In [ ]:
# drop all the columns other than "ingredient" and "label" column
columns_drop = ['index', 'categorize', 'recipe_name', 'tags', 'avg_score', 'difficulty','rating_count', 'calorie', 
                'preparation_time','comment_user', 'recipe_url']
df_certain_countries = df_certain_countries.drop(columns_drop, axis = 1)

# preprocess the ingredient column
distinct_ingredients = ingredients_preprocess(df_certain_countries)

# One hot encoding of the ingredients
df_certain_countries['ingredient'] = df_certain_countries.ingredient.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df_countries = pd.DataFrame(data = vectorizer.fit_transform(df_certain_countries.ingredient.tolist()), columns = distinct_ingredients)
new_df_countries['label'] = df_certain_countries.label

# dummy for the label column
new_df_countries = pd.get_dummies(new_df_countries)

In [ ]:
# apply 
new_df_countries

In [ ]:
test = new_df_countries.iloc[:,]
test = test.iloc[:4]

In [ ]:
country_label = list(new_df_countries.iloc[:,-19:].columns)
#contry_label.remove('label_deutschland')

# record the start time
time1 = time.time()
        
# apply subgroup discovery 
target = ps.BinaryTarget ('label_usaoderkanada', True)
searchspace = ps.create_selectors(test, ignore = country_label)
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=20, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)

# record the end time
time2 = time.time()
time_diff = (time2-time1)/60
print('it took ' + str(time_diff) + 'miniutes to execute the subgroup disc')